# Entrenamiento de las redes profundas

Entrenar una red con $10$ capas o más, y miles de conexiones, es difícil. Algunos problemas que pueden ocurrir son:

* Gradientes que desaparecen o explotan: los gradientes de la función de activación pueden reducirse a cero o pueden crecer demasiado en el proceso de propagación hacia atrás, causando problemas en el entrenamiento de las capas más profundas en la red.


* Cantidad insuficiente de datos para una red tan grande, o es demasiado difícil etiquetear todos los datos.


* Entrenamiento muy lento.


* Riesgo de *overfitting*

## Gradientes inestables

En una red profunda muy amenudo los gradientes se reducen cuando avanzamos hacia las capas más profundas (más lejanas de la salida). Entonces la actualización de los pesos en las primeras capas es muy lenta, y la red nunca llega a una solución óptima. Esto se llama el problema de gradientes que se desvanecen (*vanishing gradients*).

En algunas arquitecturas (redes recurrentes) el opuesto puede pasar: los gradientes crecen enormemente. Este es el problema de gradientes que explotan (*exploding gradients*).

En general, las distintas capas en una red profunda aprenden con tasas muy diferentes.

En Glorot & Bengio (2010) identificaron algunas razones por tener el problema de gradientes que se desvanecen:

* Consideramos una red con funciones de activación sigmoide y inicialización de los pesos con una Gaussiana de promedio $0$ y desviación estandar $1$.


* La varianza de las salidas de cada capa es mucho mayor que la varianza de las entradas de cada capa. En la pase hacia adelante (*forward pass*) la varianza sigue creciendo después de cada capa, hasta que la función de activación se satura en la capa final.


* Si la función de activación sigmoide es cerca a $0$ o $1$, los gradientes son muy pequeños, así que no hay mucho que se puede propagar hacia atrás. Además, el gradiente está diluido en cada capa en el algoritmo de *backpropagation*.

Se puede resolver el problema cambiando la inicialización de los pesos. Primero, hay que definir un par de números:

* *fan*$_{in}$: número de entradas en una capa.
* *fan*$_{out}$: número de neuronas en una capa.
* *fan*$_{avg}$: promedio de los valores arriba.

Incialización de Glorot (o Xavier):

1. usar una distribución normal con promedio $0$ y varianza $\sigma^2 = 1/\text{fan}_{avg}$.


2. Distribución uniforme entre $-r$ y $+r$ con $r=\sqrt{3/\text{fan}_{avg}}$.

Ahora hay varias opciones, dependiende de la función de activación.

![](figures_redes_profundas/table11-1.png)

Por defecto, Keras ocupa Glorot, pero se puede cambiar:

`keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")`

Para usar He inicialización con una distribución uniforme pero basada en *fan*$_{avg}$ en vez de *fan*$_{in}$:

```
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)
```

### Funciones de activación que no se saturan

El uso de la función sigmoide fue inspirado por las neuronas biológicas. Pero el problema de los *vanishing gradients* en redes profundas impulsó el uso de otras funciones, como por ejemplo la función ReLU (que no se satura para valores positivos).

El problema con los ReLU es que durante el entrenamiento algunas neuronas pueden "morirse": sus salidas son siempre $0$. Este ocurre cuando la suma ponderada de entradas a la neurona es negativo para todas las instancias. El ReLU tiene gradiente cero para valores negativos, así que los pesos no cambian más y la neurona se muere.

Hay una variante del ReLU que se llama *leaky ReLU*:

$$\text{LeakyReLU}_{\alpha}(z) = \text{max}(\alpha z, z)$$

El hiperparámetro $\alpha$ determina cuanto "filtra" la función: es la pendiente para $z < 0$ y típicamente es igual a $0.01$. Así las neuronas nunca se mueren, aunque pueden existir en un "coma" prolongada...

![](figures_redes_profundas/fig11-2.png)

En Clevert et al. (2015) propusieron una nueva función de activación que se llama *exponential linear unit* (ELU) que parece funciona mejor que cualquier variante del ReLU.

$$\text{ELU}_{\alpha} (z) = \begin{cases} \alpha (\exp(z) - 1) & z < 0 \\ z & z \geq 0 \end{cases}$$

![](figures_redes_profundas/fig11-3.png)

Diferencias con el ELU, comparada con el ReLU.

* Toma valores negativos cuando $z < 0$, que ayuda tener la salida promedia más cerca a $0$ y alivia el problema de *vanishing gradients*. El parámetro $\alpha$ es tal que $\text{ELU}(z) \to \alpha$ cuando $z \to -\infty$.


* Tiene un gradiente no cero para $z < 0$ que elimina el problema de neuronas muertas.


* Si $\alpha = 1$ la función es suave en todos puntos, incluyendo $z=0$, que ayuda en descenso por gradiente.

La desventaja que es que es más lento usar una red con ELU en vez de ReLU.

En Klambauer et al. (2017) descubrieron que si usamos una función que se llama SELU (*scaled ELU*) en un red con solamente capas densas y donde todas las capas ocultas ocupan SELU, la red muestra **auto normalización**: la salida de cada capa tiende a preservar un promedio de $0$ y una desciación estandar de $1$ durante el entrenamiento. Este resuelve por completo el problema de *exploding/vanishing gradients*.

Hay condiciones para asegurar auto normalización:

* Los *features* de las entradas deben ser estandarizados (promedio $0$, desviación estandar $1$)


* Cada capa oculta debe ser incializada con inicialización de LeCun normal.


* La arquitectura de la red debe ser secuencial. Si es recurrente o si tiene conexiones que saltan capas, como en el caso de *Wide & Deep*, auto normalización no está garantizada.


* Auto normalización está garantizada solamente si las capas son densas (completamente conectadas). Algunos investigadores han notado que SELU puede mejorar el rendimiento de redes convolucionales también.

En resumen, podemos ordenar las funciones de activación, de la mejor hasta la peor:

1. SELU
2. ELU
3. Leaky ReLU (y variantes)
4. ReLU
5. tanh
6. Logística

Muchas librerias ocupan ReLU por defecto, y están optimizadas para usar esa función, así que van a correr más rápido con esa.

Para implementar LeakyRelU:

```
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    [...]
])
```

Para SELU:

```
layer = keras.layers.Dense(10, activation="selu", 
                           kernel_initializer="lecun_normal")
```

## Normalización por lotes (*batch normalization*)

Desarrollado por Ioffe y Szegedy (2015). Esta técnica es también para evitar el problema de los *vanishing/exploding gradients*.

La idea es poner una operación en el modelo justo antes o después de la función de activación de cada capa oculta. Esta operación centra y normaliza las entradas, y después aplica un escalamiento y un desplazamiento al resultado.

Algoritmo:

1. $\boldsymbol{\mu}_B = \frac{1}{m_B}\sum_{i=1}^{m_B} \boldsymbol{x}^{(i)}$


2. $\boldsymbol{\sigma}_B^2 = \frac{1}{m_B} \sum_{i=1}^{m_B} \left( \boldsymbol{x}^{(i)} - \boldsymbol{\mu}_B \right)^2$


3. $\hat{\boldsymbol{x}}^{(i)} = \frac{\boldsymbol{x}^{(i)}-\boldsymbol{\mu}_B}{\sqrt{\boldsymbol{\sigma}_B^2 + \epsilon}}$


4. $\boldsymbol{z}^{(i)} = \boldsymbol{\gamma} \otimes \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}$

donde:

* $\boldsymbol{\mu}_B$ es el vector de promedios de las entradas, evaluado sobre todo el *mini-batch*.


* $\boldsymbol{\sigma}_B$ es el vector de desviaciones estandares de las entradas, evaluado sobre todo el *mini-batch*.


* $m_B$ es el número de instancias en el *mini-batch*.


* $\hat{\boldsymbol{x}}^{(i)}$ es el vector de entradas centradas y normalizadas para instancia $i$.


* $\boldsymbol{\gamma}$ es el vector de parámetros de escala para la capa (un parámetro de escala por entrada).


* $\otimes$ representa multiplicación elemento-por-elemento.


* $\boldsymbol{\beta}$ es el vector de desplazamientos para la capa.


* $\epsilon$ es un parámetro pequeño $(\sim 10^{-5})$ para evitar división por cero.


* $\boldsymbol{z}^{(i)}$ es la salida de la operación de *batch normalization* (BN).

En el momento de probar el modelo (usar el conjunto de prueba) típicamente no tenemos lotes de instancias, así que no podemos aplicar BN como está descrito arriba.

Keras hace lo siguiente:

1. Los vectores $\boldsymbol{\gamma}$ y $\boldsymbol{\beta}$ están aprendidos por *backpropagation*.

2. Los vectores $\boldsymbol{\mu}$ y $\boldsymbol{\sigma}$ están estimados durante el proceso de entrenamiento usando un promedio móvil exponencial.

Los autores del método demostraron que era posible usar funciones de activación que se saturan en una red profunda si BN está implementado.

#### Implementación de BN en Keras

Volvemos al ejemplo de Fashion MNIST.

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

Cada capa de BN agrega $4$ parámetros por entrada: $\boldsymbol{\gamma}$, $\boldsymbol{\beta}$, $\boldsymbol{\mu}$ y $\boldsymbol{\sigma}$. Por ejemplo, la primera capa agrega $4 \times 784 = 3136$ parámetros.

Los parámetros $\boldsymbol{\mu}$ y $\boldsymbol{\sigma}$ no están afectados por *backpropagation* así que Keras se refiere a esos parámetros como "no entrenable" (hay $(3136+1200+400)/2 = 2368$ parámetros así).

In [7]:
#Parámetros en la primera capa BN
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

Hay varios hiperparámetros asociados a BN. Dos de estos son:

* `momentum`: usado en la actualización de los promedios móviles. Con un nuevo vector (de promedios o desviaciones estandares) $\boldsymbol{v}$ el promedio está actualizado según $\hat{\boldsymbol{v}} \leftarrow \hat{\boldsymbol{v}} \times \text{momentum} + \boldsymbol{v} \times (1-\text{momentum})$. Un valor bueno es muy cerca a $1$.


* `axis`: determina cual eje debería ser normalizado. Por defecto es igual a $-1$ (normaliza el último eje usando promedios y DE calculado de los otros ejes).

*Batch normalization* es tan común en redes profundas que muchas veces las capas de BN no aparecen en los diagramas de los modelos, aunque están incluidos. La suposición es que hay una capa de BN después de cada capa oculta.

[Zhang et al. (2019)](https://arxiv.org/abs/1901.09321) utilizaron una técnica nueva para la inicialización de los pesos, y lograron entrenar una red con 10.000 capas sin BN.

### *Gradient Clipping*

Otra técnica para reducir el problema de gradientes que se explotan. La idea es simplemente "cortar" los gradientes si exceden un umbral. Es más usado para redes recurrentes.

`optimizer = keras.optimizers.SGD(clipvalue=1.0)`
`model.compile(loss="mse", optimizer=optimizer)`

Este puede cambiar la dirección del gradiente. Por ejemplo si el vector original es `[0.9, 100.0]`, después será `[0.9, 1.0]`.

Se puede evitar ese problema usando `clipnorm` que corta el gradiente si la norma $\mathcal{l}_2$ excede el umbral. Por ejemplo `[0.9, 100.0]` después será `[0.00899964, 0.9999595]`.

## Otros optimizadores

El entrenamiento de una red profunda puede ser muy lento. Hasta ahora hemos visto $3$ maneras de acelerar el proceso:

* Aplicar una buena estrategia de inicialización.


* Utilizar una buena función de activación.


* *Batch normalization*

También existe la opción de usar una parte de un modelo ya entrenado (ver el libro).

Otra opción es cambiar el optimizador...

### Optimización momentum

En este método los gradientes están usados para *acelerar* el movimiento en el espacio de la función de costo. Hay un nuevo hiperparámetro $\beta$ que corresponde a una "fricción" para frenar el movimiento. $\beta = 0$ (fricción alta), $\beta = 1$ (sin fricción).

Algoritmo:

1. $\boldsymbol{m} \leftarrow \beta\boldsymbol{m} - \eta \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$

2. $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \boldsymbol{m}$

Implementación:

`optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)`

### Gradiente acelerado de Nesterov (NAG)

En esta variante el gradiente está medida no en la posición local $\boldsymbol{\theta}$ sino que un poco más adelante en la dirección del momentum, en $\boldsymbol{\theta} + \beta \boldsymbol{m}$.

1. $\boldsymbol{m} \leftarrow \beta \boldsymbol{m} - \eta \nabla_{\boldsymbol{\theta}} J (\boldsymbol{\theta} + \beta \boldsymbol{m})$

2. $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \boldsymbol{m}$

![](figures_redes_profundas/fig11-6.png)

Implementación:

`optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)`

### AdaGrad

Algoritmo:

1. $\boldsymbol{s} \leftarrow \boldsymbol{s} + \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \otimes \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$

2. $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \oslash \sqrt{\boldsymbol{s} + \epsilon}$

El primer paso acumula los gradientes cuadrados en los elementos del vector $\boldsymbol{s}$. Es equivalente a:

$$s_i \leftarrow s_i + \left( \frac{\partial J(\boldsymbol{\theta})}{\partial \theta_i} \right)^2$$

Si la función de costo es muy inclinada en la dirección que corresponde al parámetro $\theta_i$, el elemento $s_i$ crecerá en cada iteración.

El segundo paso aplica un factor al vector del gradiente ($\oslash$ significa división elemento-por-elemento). Es equivalente a:

$$\theta_i \leftarrow \theta_i - \eta \left( \frac{\partial J(\boldsymbol{\theta})}{\partial \theta_i} \right) \frac{1}{\sqrt{s_i + \epsilon}}$$

Entonces la tasa de aprendizaje está reducida más rápidamente en las direcciones más inclinadas. Así que tiene una tasa de aprendizaje **adaptativa**.

![](figures_redes_profundas/fig11-7.png)

De hecho AdaGrad funciona bien para modelos simples, pero no es apto para una red profunda: la tasa de aprendizaje está reducida tanto que el algoritmo no llega al mínimo. Pero ayuda en entender los otros optimizadores adaptativos.

### RMSProp

El problema de AdaGrad es que reduce su velocidad demasiado. RMSProp evita este problema por usar solamente los gradientes de las iteraciones más recientes (en vez de todos desde el comienzo del entrenamiento).

Algoritmo:

1. $\boldsymbol{s} \leftarrow \beta \boldsymbol{s} + (1 - \beta) \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \otimes \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$

2. $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \eta \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \oslash \sqrt{\boldsymbol{s} + \epsilon}$

El parámetro $\beta$ (tasa de decaimiento) típicamente es igual a $0.9$.

Implementación:

`optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)`

`rho` aquí corresponde a $\beta$ arriba.

### Optimización de Adam y Nadam

Adam (*adaptive moment estimation*) combina las ideas de optimización de momentum y RMSProp.

* Como optimización de momentum, sigue un promedio que decae exponencialmente de gradientes pasados.
* Como optimización RMSProp, sigue un promedio que decae exponencialmente de gradientes cuadrados pasados.

Algoritmo:

1. $\boldsymbol{m} \leftarrow \beta_1 \boldsymbol{m} - (1 - \beta_1) \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$


2. $\boldsymbol{s} \leftarrow \beta_2 \boldsymbol{s} + (1 - \beta_2) \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) \otimes \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$


3. $\hat{\boldsymbol{m}} \leftarrow \frac{\boldsymbol{m}}{1-\beta_2^t}$


4. $\hat{\boldsymbol{s}} \leftarrow \frac{\boldsymbol{s}}{1-\beta_2^t}$


5. $\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} + \eta \hat{\boldsymbol{m}} \oslash \sqrt{\hat{\boldsymbol{s}} + \epsilon}$

$t$ significa el número de iteración.

$\boldsymbol{m}$ y $\boldsymbol{s}$ están inicializados en $0$, así que tienen un sesgo hacia $0$ al principio del entrenamiento. Pasos 3 y 4 son para dar un *boost* a estos vectores cuando el entrenamiento comienza.

Típicamente $\beta_1 = 0.9$ y $\beta_2 = 0.999$.

Implementación:

`optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)`

#### Variantes

**AdaMax**: reemplaza paso 2 por $\boldsymbol{s} \leftarrow \max(\beta_2 \boldsymbol{s}, \nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta})$.

**Nadam**: Adam + Nesterov

Todas las técnicas de optimización que hemos visto utilizan derivadas parciales de primer orden (**Jacobianos**).

Hay técnicas en la literatura que aplican derivadas parciales de segundo orden (**Hessianas**).

El problema es que hay $n^2$ hessianas por salida (donde $n$ es el número de parámetros), comparado con $n$ jacobianos.

Para una red profunda hay miles de parámetros, así que muchas veces las hessianas no caben en la memoria...

### Planificación de la tasa de aprendizaje

![](figures_redes_profundas/fig11-8.png)

* Ley de potencia: $\eta(t) = \eta_0 / (1+t/s)^c$. Típicamente $c=1$. Después de $s$ iteraciones $\eta = \eta_0/2$, después de $2s$ iteraciones $\eta = \eta_0/3$ etc.


* Exponencial: $\eta(t) = \eta_0 0.1^{t/s}$. $\eta$ reduce por un factor de $10$ cada $s$ iteraciones.


* Constante por partes: usar $\eta_0 = 0.1$ (por ejemplo) para $5$ iteraciones, después $\eta_1 = 0.001$ para $50$ iteraciones, etc.


* Rendimiento: medir el error de validación cada $N$ iteraciones, reducir la tasa de aprendizaje por un factor $\lambda$ cuando el error acaba de bajar.


* *1cycle*: Smith (2018). Aumenta la tasa inicial linealmente de $\eta_0$ a $\eta_1$, en la primera mitad del entrenamiento. Después reduce $\eta$ hasta $\eta_0$ en la segunda mitad. Elegimos $\eta_1$ usando el mismo método descrito antes para encontrar la tasa óptima. Si usamos momentum, comenzamos con un momentum alto ($0.95$), se reduce a $0.85$ linealmente en la primera mitad, y sube al máximo de nuevo en la segund mitad.

Implementación:

* Ley de potencia: `optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)`. `decay` es la inversa de $s$, y $c=1$.

* Exponencial: se puede definir una función que retorna otra función que define la tasa de aprendizaje.

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn
    
exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])

`LearningRateScheduler` actualizará `learning_rate` al principio de cada época.

* Constante por partes: se puede crear una función, como lo que hicimos para la exponencial.

In [ ]:
def piecewise_constat_fn(epoch):
    if (epoch < 5):
        return 0.01
    elif (epoch < 15):
        return 0.005
    else:
        return 0.001

* Rendimiento: se puede usar el *callback* `ReduceLROnPlateau`

`lr_scheduler = keras.callbacks.ReduceLROnPlateua(factor=0.5, patience=5)`

Otra opción es usar los *schedules* disponibles en `keras.optimizers.schedules`:

In [ ]:
s = 20 * len(X_train) // 32 #número de iteraciones en 20 épocas (lote = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

Para implementar *1cycle* también se puede crear un *callback* para actualizar la tasa de aprendizaje en cada iteración.

## Regularización

Vamos a ver algunas formas de regularizar redes profundas:

* Regularización $\mathcal{l}_1$ y $\mathcal{l}_2$.


* *Dropout*


* *Max-norm*

### Regularización $\mathcal{l}_1$ y $\mathcal{l}_2$

Para $\mathcal{l}_2$ podemos usar:

In [ ]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

Esto aplicará regularización $\mathcal{l}_2$ con un factor de $0.01$ a los pesos de la capa. La función `l2()` retorna un regularizador que está llamado en cada iteración durante el entrenamiento, y después está sumado a la función de costo final.

Para $\mathcal{l}_1$: `keras.regularizers.l1()`. En este caso varios pesos podrían acercarse al valor $0$.

Para ambas regularizaciones: `keras.regularizers.l1_l2()`

Un truco para no tener que repetir los argumentos a `Dense` en cada capa:

In [ ]:
from functools import partial

In [ ]:
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
                     kernel_initializer="glorot_uniform")
])

### *Dropout*

Este es un método muy popular de regularizar redes profundas.

La idea es: en cada iteración de entrenamiento cada neurona (excluyendo las neuronas de salida) tiene una probabilidad $p$ de estar desactivada (*dropped out*).

La tasa de *dropout*, $p$, tiene un valor típicamente entre $10\%$ y $50\%$ (menor en redes recurrentes, mayor en redes convolucionales).

![](figures_redes_profundas/fig11-9.png)

Se puede interpretar el uso de *dropout* como el entrenamiento de muchas redes diferentes (pero relacionadas). La red final es como el ensamble de todas estas redes.

Un detalle importante: supongamos que $p = 0.5$ ($50\%$). Durante el proceso de evaluación una neurona será conectada al doble del número de neuronas conectadas en el entrenamiento. 

Por eso, hay que multiplicar los pesos en las conexiones de entrada da cada neurona por $0.5$. Si no, cada neurona recibirá una señal dos veces mayor (en evaluación) comparado con el entrenamiento, y es muy probable que no tendrá un rendimiento bueno.

En general, hay que multiplicar cada peso de entrada por la *keep probability* $(1-p)$ después del entrenamiento (Keras se encarga de esto).

In [6]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [8]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [15]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5757 - accuracy: 0.8037 - val_loss: 0.3829 - val_accuracy: 0.8610
Epoch 2/2
1719/1719 [==============================] - 11s 7ms/step - loss: 0.4213 - accuracy: 0.8468 - val_loss: 0.3523 - val_accuracy: 0.8728


**Ojo**: *dropout* está activado solamente durante el entrenamiento, así comparando la perdida de entrenamiento con la de validación podría ser engañoso... Un modelo podría *overfit* los datos de entrenamiento, pero tener perdidas similares entre entrenamiento y validación. Hay que evaluar la perdida de entrenamiento después, sin *dropout*.

Para usar *dropout* con una red auto normalizada (usando SELU) hay que usar *alpha dropout*. *Dropout* normal destruye la auto normalización.

### *Dropout* de Monte Carlo (MC)

Con MC *Dropout* se puede mejorar el rendimiento de un modelo ya entrenado con *dropout*:

In [16]:
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range(100)])
y_proba = y_probas.mean(axis=0)

Hacemos $100$ predicciones en el conjunto de prueba, con `training=True` para asegurar que la capa de *dropout* está activa, y después combinamos las predicciones.

Ya que *dropout* está activo, todas las predicciones son diferentes.

`predict()` retorna una matriz con una fila por instancia y una columna por clase. Con $10000$ instancias en el conjunto de prueba y $10$ clases, la matriz tendrá una forma de $10000 \times 10$.

Combinamos $100$ matrices así, entonces:

In [11]:
y_probas.shape

(100, 10000, 10)

Calculando el promedio con `axis=0` obtenemos:

In [12]:
y_proba.shape

(10000, 10)

Este es lo que tendríamos haciendo una predicción con el modelo para cada instancia en el conjunto de prueba.

El promedio con *dropout* es una estimación de Monte Carlo (muestra aleatoria) que es típicamente más confiable que el resultado de una sola predicción sin *dropout*.

In [17]:
np.round(model.predict(X_test_scaled[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.22, 0.  , 0.77]],
      dtype=float32)

In [20]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.31, 0.  , 0.67]],
      dtype=float32)

Con el resultado de MC *dropout* tenemos una estimación más razonable, y nos da más información sobre posibles confusiones del modelo.

Podemos medir la incertidumbre con la desviación estandar en las predicciones de la versión MC *dropout*.

In [21]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.21, 0.  , 0.22]],
      dtype=float32)

In [26]:
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8577

In [27]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8568

Si el modelo incluye otras capas que tienen un comportamiento especial durante el entrenamiento (e.g. `BatchNormalization`) no podemos usar modo `training=True`. Tenemos que reemplazar las capas de `Dropout` por capas de `MCDropout` definidas en el código abajo:

In [ ]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

### Regularización *max-norm*

Para cada neurona, restringe los pesos de las conexiones de entrada tal que $|| \boldsymbol{w} ||_2 \leq r$ donde $r$ es el parámetro *max-norm* y $|| \cdot ||_2$ es la norma $\mathcal{l}_2$.

Está implementado por calcular $|| \boldsymbol{w} ||_2$ después de cada iteración y modificar $\boldsymbol{w}$ si es necesario con:

$\boldsymbol{w} \leftarrow \boldsymbol{w} \frac{r}{|| \boldsymbol{w} ||_2}$

Implementación en Keras:

In [ ]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                   kernel_constraint=keras.constraints.max_norm(1.))

Se puede definir funciones para implementar restricciones customizadas. También podemos restringir los *bias* con `bias_constraint`.

La función `max_norm()` tiene un argumento `axis` (por defecto igual a $0$). Para capas convolucionales hay que aplicar el `axis` correctamente.